# Capital Gains Distributions 2021

## Prepare for Data Scraping

### Import Libraries

In [1]:
import datetime
import numpy as np
import os
import pandas as pd
import pdfplumber
import random
import re
import requests
import time as t


from bs4 import BeautifulSoup
from dateutil.parser import parse
from pprint import pprint
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

### ChromeDriver Installation

We have included chromedriver for Windows + Chrome Version 86 in this zip file, but for further information and instructions on how to install, please refer to the below link:
* https://chromedriver.chromium.org/downloads

If your system requires a different version, please download it and replace the .exe we have provided.

For supplemental instructions, please see the README.me file.

### Global Variables

In [2]:
data_pdf_path = 'data_pdf'

### Global Functions

#### Soup Retrieval

In [3]:
def get_page_and_parse(url):

    user_agent = {'User-Agent':'Mozilla/5.0'}

    API_response_html = requests.get(url, headers=user_agent)
    
    page_soup = BeautifulSoup(API_response_html.content, 'html.parser')
    page_response = API_response_html

    return page_soup, page_response

#### Download PDF

In [4]:
def download_file(url):
    local_filename = url.split('/')[-1]

    with requests.get(url) as r:
        with open(os.path.join(data_pdf_path, local_filename), 'wb') as f:
            f.write(r.content)
            
    return local_filename

#### Soup Retrieval (Selenium + ChromeDriver)

In [5]:
def chrome_driver(url, xpath=None, scroll=True, scrolls=10, scroll_hop=2000):
    
    path = os.path.join('chrome_driver', 'chromedriver')
    s = Service(path)
    driver = webdriver.Chrome(service=s)

    # Navigate to URL
    driver.get(url)
    t.sleep(5)
    retrieval_timestamp = datetime.datetime.now()

    # Scroll the Desired Table
    if scroll==True:
        element = driver.find_element(By.XPATH, xpath)
        for i in range(scrolls):
            driver.execute_script(f"arguments[0].scrollBy(0, {scroll_hop})", element)
            random_sleep_time = int(random.uniform(1,4))
            t.sleep(random_sleep_time)

    # Convert HTML to Soup
    html = driver.page_source
    page_soup = BeautifulSoup(html, 'html.parser')
    driver.close()
    
    return page_soup

### Create Final DataFrame

In [6]:
columns = ['Fund Family',
           'Fund Name',
           'Ticker',
           'Short Term Capital Gain',
           'Long Term Capital Gain',
           'Capital Gain Format',
           'Distribution Date',
           'Timestamp']

df_final = pd.DataFrame(columns=columns)
df_final.dtypes

Fund Family                object
Fund Name                  object
Ticker                     object
Short Term Capital Gain    object
Long Term Capital Gain     object
Capital Gain Format        object
Distribution Date          object
Timestamp                  object
dtype: object

## Parsing Scripts by Firm

### T. Rowe Price

#### _Get Soup_

In [7]:
url = 'https://www.troweprice.com/personal-investing/resources/planning/tax/dividend-distributions/mutual-funds/preliminary-year-end-distributions.html'
page_soup, page_response = get_page_and_parse(url)

#### _Parse Data_

In [8]:
fund_series = ''

fund_family = 'T. Rowe Price'
fund_name = []
ticker = []
capital_gain_format = 'Per Share'
short_term = []
long_term = []
distribution_date = []

for g, div in enumerate(page_soup.find_all("div", {"class": "accordion-content parsys"})):

    date_section = div.find('span').text
    if re.search('Payment Date:', date_section):
        date_parsed = parse(date_section.split(':')[-1].strip())
        date_formatted = datetime.date.strftime(date_parsed, '%m/%d/%y')
        current_date = date_formatted
    
    for i, table in enumerate(div.find_all('table')):
        for j, row in enumerate(table.find_all('tr')):
            for k, data in enumerate(row.find_all('td')):
                if i==0 and j==0 and k==0:
                    fund_series = ' '.join(data.text.split())
                    continue
                elif j < 3:
                    continue
                elif k==0:
                    fund_name.append("{} - {}".format(fund_series.replace('\n', ''), data.text.replace('\n', '')))
                    distribution_date.append(current_date)
                elif k==1:
                    fund_name[-1] = f'{fund_name[-1]} ({data.text.strip()})'
                    ticker.append(data.text.strip())
                elif k==4:
                    num = data.text.strip().replace('$','')
                    num_flt = float(num) if num.replace('.','').isdecimal() else np.NaN
                    short_term.append(num_flt)
                elif k==5:
                    num = data.text.strip().replace('$','')
                    num_flt = float(num) if num.replace('.','').isdecimal() else np.NaN
                    long_term.append(num_flt)

#### _Create DataFrame_

In [9]:
df = pd.DataFrame({'Fund Name': fund_name})
df['Fund Family'] = fund_family
df['Distribution Date'] = distribution_date
df['Short Term Capital Gain'] = short_term
df['Long Term Capital Gain'] = long_term
df['Capital Gain Format'] = capital_gain_format
df['Ticker'] = ticker
df['Timestamp'] = datetime.datetime.now()

df = df[columns]
df_final = df_final.append(df).reset_index(drop=True)

df.head()

,Fund Family,Fund Name,Ticker,Short Term Capital Gain,Long Term Capital Gain,Capital Gain Format,Distribution Date,Timestamp
0,T. Rowe Price,T. Rowe Price Domestic Stock Funds - Dividend ...,PRDGX,NaN,0.89,Per Share,12/14/21,2021-12-02 20:12:13.973473
1,T. Rowe Price,T. Rowe Price Domestic Stock Funds - Equity In...,PRFDX,0.36,1.95,Per Share,12/14/21,2021-12-02 20:12:13.973473
2,T. Rowe Price,T. Rowe Price Domestic Stock Funds - Equity In...,PREIX,0.07,0.47,Per Share,12/14/21,2021-12-02 20:12:13.973473
3,T. Rowe Price,T. Rowe Price Domestic Stock Funds - Extended ...,PEXMX,NaN,5.06,Per Share,12/14/21,2021-12-02 20:12:13.973473
4,T. Rowe Price,T. Rowe Price Domestic Stock Funds - Real Esta...,TRREX,NaN,2.80,Per Share,12/14/21,2021-12-02 20:12:13.973473


### PIMCO

#### _Download PDF_

In [10]:
pdf_url = 'https://bit.ly/3D4Up1U'

pdf_downloaded = download_file(pdf_url)
retrieval_timestamp = datetime.datetime.now()

#### _Parse & Collect Data_

In [11]:
text = ''

fund_family = 'PIMCO'
fund_name = []
short_term = []
long_term = []
capital_gain_format = 'Per Share'
distribution_date = []

with pdfplumber.open(os.path.join(data_pdf_path, pdf_downloaded)) as pdf:
    for page_num in range(len(pdf.pages)):
        page = pdf.pages[page_num]
        text += page.extract_text()
        
for line in text.split('\n'):
    
    if line.startswith('Payment Date:'):
        payment_date_parsed = parse(line.split(':')[1].strip())
        payment_date_formatted = datetime.date.strftime(payment_date_parsed, '%m/%d/%y')
        
    elif line.startswith('PIMCO ') and re.search('\$', line):
        
        row = line.split('$')
        row = [x.strip() for x in row]
        row[2] = row[2].split(' ')[0]
        
        fund_name.append(row[0].strip())
        short_term.append(float(row[1].strip()) if row[1].replace('.','').strip().isdecimal() else np.NaN)
        long_term.append(float(row[2].strip()) if row[2].replace('.','').strip().isdecimal() else np.NaN)
        distribution_date.append(payment_date_formatted)

#### _Create DataFrame_

In [12]:
df = pd.DataFrame({'Fund Name': fund_name})
df['Fund Family'] = fund_family
df['Distribution Date'] = distribution_date
df['Short Term Capital Gain'] = short_term
df['Long Term Capital Gain'] = long_term
df['Capital Gain Format'] = capital_gain_format
df['Ticker'] = None
df['Timestamp'] = datetime.datetime.now()

df = df[columns]
df_final = df_final.append(df).reset_index(drop=True)

df.head()

,Fund Family,Fund Name,Ticker,Short Term Capital Gain,Long Term Capital Gain,Capital Gain Format,Distribution Date,Timestamp
0,PIMCO,PIMCO All Asset All Authority Fund,None,NaN,NaN,Per Share,12/08/21,2021-12-02 20:12:23.395820
1,PIMCO,PIMCO All Asset Fund,None,NaN,NaN,Per Share,12/08/21,2021-12-02 20:12:23.395820
2,PIMCO,PIMCO California Intermediate Municipal Bond Fund,None,NaN,NaN,Per Share,12/08/21,2021-12-02 20:12:23.395820
3,PIMCO,PIMCO California Municipal Bond Fund,None,NaN,NaN,Per Share,12/08/21,2021-12-02 20:12:23.395820
4,PIMCO,PIMCO California Municipal Intermediate Value ...,None,NaN,NaN,Per Share,12/08/21,2021-12-02 20:12:23.395820


### Franklin Templeton

#### _Get Soup_

In [13]:
url = 'https://franklintempletonprod.widen.net/s/l9sr2x8pd8/2021-capital-gain-indications'
page_soup = chrome_driver(url, '//*[@id="viewerContainer"]')
retrieval_timestamp = datetime.datetime.now()

#### _Parse & Collect Data_

In [14]:
fund_family = 'Franklin Templeton'

fund_name = []
distribution_date = []
short_term = []
long_term = []
capital_gain_format = 'Per Share Percent'

counter = 1

for i, text in enumerate(page_soup.find_all('div', {'class': 'textLayer'})[:6]):
    for j, row in enumerate(text.find_all('span')):
        counter = 1 if counter > 5 else counter
        if (i == 0 and j < 37) or (i == 5 and j > 114) or row.text in ['Hedged) ', 'Market Fund ']:
            pass
        else:
            row_text = row.text.strip()
            
            if counter==1:
                fund_name.append(row_text)
            elif counter==2:
                distribution_date.append(datetime.date.strftime(parse(row_text), '%m/%d/%y')) if row_text != 'See Note 1' else distribution_date.append(None)
            elif counter==3:
                row_text_flt = float(row_text.replace('%', '')) / 100
                short_term.append(row_text_flt)
            elif counter==4:
                row_text_flt = float(row_text.replace('%', '')) / 100
                long_term.append(row_text_flt)
            
            counter += 1

#### _Create DataFrame_

In [15]:
df = pd.DataFrame({'Fund Name': fund_name})
df['Fund Family'] = fund_family
df['Distribution Date'] = distribution_date
df['Short Term Capital Gain'] = short_term
df['Long Term Capital Gain'] = long_term
df['Capital Gain Format'] = capital_gain_format
df['Ticker'] = None
df['Timestamp'] = retrieval_timestamp

df = df[columns]
df_final = df_final.append(df).reset_index(drop=True)

df.head()

,Fund Family,Fund Name,Ticker,Short Term Capital Gain,Long Term Capital Gain,Capital Gain Format,Distribution Date,Timestamp
0,Franklin Templeton,BrandywineGLOBAL - Alternative Credit Fund,None,0.000,0.000,Per Share Percent,12/17/21,2021-12-02 20:13:11.857019
1,Franklin Templeton,BrandywineGLOBAL - Corporate Credit Fund,None,0.015,0.004,Per Share Percent,12/02/21,2021-12-02 20:13:11.857019
2,Franklin Templeton,BrandywineGLOBAL - Diversified US Large Cap Va...,None,0.016,0.182,Per Share Percent,12/02/21,2021-12-02 20:13:11.857019
3,Franklin Templeton,BrandywineGLOBAL - Dynamic US Large Cap Value ...,None,0.114,0.030,Per Share Percent,12/02/21,2021-12-02 20:13:11.857019
4,Franklin Templeton,BrandywineGLOBAL - Flexible Bond Fund,None,0.003,0.000,Per Share Percent,12/17/21,2021-12-02 20:13:11.857019


### American Century

#### _Get Soup_

In [16]:
url = 'https://www.americancentury.com/content/direct/en/insights/guidance-planning/tax-center/preparing-your-taxes/estimated-distributions.html'
page_soup = chrome_driver(url, scroll=False)
retrieval_timestamp = datetime.datetime.now()

#### _Parse & Collect Data_

In [17]:
fund_family = 'American Century'
fund_name = []
distribution_date = []
short_term = []
long_term = []

for i, table in enumerate(page_soup.find_all('table')):
    rows = table.find_all('tr')
    for tr in rows:
        cols = tr.find_all('td')
        for i, td in enumerate(cols):
            if(i == 0):
                fund_name.append(td.text)
            elif(i == 5):
                distribution_date.append(datetime.date.strftime(parse(td.text), '%m/%d/%y'))
            elif(i == 3):
                short_term.append(float(td.text))
            elif(i == 4):
                long_term.append(float(td.text))

#### _Create DataFrame_

In [18]:
df = pd.DataFrame({'Fund Name': fund_name})
df['Fund Family'] = fund_family
df['Distribution Date'] = distribution_date
df['Short Term Capital Gain'] = short_term
df['Long Term Capital Gain'] = long_term
df['Ticker'] = None
df['Timestamp'] = retrieval_timestamp
df['Capital Gain Format'] = 'Per Share'

df = df[columns]
df_final = df_final.append(df).reset_index(drop=True)

df.head()

,Fund Family,Fund Name,Ticker,Short Term Capital Gain,Long Term Capital Gain,Capital Gain Format,Distribution Date,Timestamp
0,American Century,AC Alternatives® Market Neutral Value Fund,None,0.0000,0.0000,Per Share,12/21/21,2021-12-02 20:13:42.696231
1,American Century,Balanced Fund,None,1.9205,2.3251,Per Share,12/21/21,2021-12-02 20:13:42.696231
2,American Century,California High-Yield Municipal Fund,None,0.0000,0.0000,Per Share,12/07/21,2021-12-02 20:13:42.696231
3,American Century,California Intermediate-Term Tax-Free Bond Fund,None,0.0000,0.0094,Per Share,12/07/21,2021-12-02 20:13:42.696231
4,American Century,Core Plus Fund,None,0.0361,0.0683,Per Share,12/07/21,2021-12-02 20:13:42.696231


### American Funds/Capital Group

#### _Get Soup_

In [19]:
url = 'https://www.capitalgroup.com/advisor/tax/2021-year-end-distributions.html'
soup, page_response = get_page_and_parse(url)
retrieval_timestamp = datetime.datetime.now()

#### _Parse & Collect Data_

In [20]:
fund_family = 'American Funds'
fund_name = []
distribution_date = []
short_term = []
long_term = []

for i, table in enumerate(soup.find_all('table')[:4]):
    for j, row in enumerate(table.find_all('tr')[4:]):
        for k, para in enumerate(row.find_all('p')):
            # print(j, k, para)
            if k == 0:
                fund_name.append(para.text)
            elif k== 2:
                distribution_date.append(datetime.date.strftime(parse(para.text), '%m/%d/%y'))
            elif k== 3:
                num_range = para.text.replace(u'\xa0', '').replace('%', '').split('-')
                num_range_flt = [int(num.strip()) if num.strip().isnumeric() == True else np.NaN for num in num_range]
                long_term.append(np.mean(num_range_flt) / 100)
            elif k== 4:
                num_range = para.text.replace(u'\xa0', '').replace('%', '').split('-')
                num_range_flt = [int(num.strip()) if num.strip().isnumeric() == True else np.NaN for num in num_range]
                short_term.append(np.mean(num_range_flt) / 100)

#### _Create DataFrame_

In [21]:
df = pd.DataFrame({'Fund Name': fund_name})

df['Fund Family'] = 'American Funds'
df['Short Term Capital Gain'] = short_term
df['Long Term Capital Gain'] = long_term
df['Capital Gain Format'] = 'Per Share Percent'
df['Ticker'] = None
df['Distribution Date'] = distribution_date
df['Timestamp'] = retrieval_timestamp

df = df[columns]
df_final = df_final.append(df).reset_index(drop=True)

df.head()

,Fund Family,Fund Name,Ticker,Short Term Capital Gain,Long Term Capital Gain,Capital Gain Format,Distribution Date,Timestamp
0,American Funds,AMCAP Fund®,None,NaN,0.030,Per Share Percent,12/16/21,2021-12-02 20:14:07.911210
1,American Funds,American Balanced Fund®,None,NaN,0.030,Per Share Percent,12/15/21,2021-12-02 20:14:07.911210
2,American Funds,American Funds Corporate Bond Fund®,None,0.005,0.005,Per Share Percent,01/03/22,2021-12-02 20:14:07.911210
3,American Funds,American Funds Developing World Growth and Inc...,None,NaN,NaN,Per Share Percent,12/22/21,2021-12-02 20:14:07.911210
4,American Funds,American Funds Emerging Markets Bond Fund®,None,NaN,0.005,Per Share Percent,01/03/22,2021-12-02 20:14:07.911210


### BlackRock

#### _Get Soup_

In [22]:
url = 'https://www.blackrock.com/us/individual/resources/tax-information/2021-distributions'
page_soup = chrome_driver(url, scroll=False)
retrieval_timestamp = datetime.datetime.now()

#### _Parse & Collect Data_

In [23]:
fund_family = 'BlackRock'
fund_name = []
distribution_date = []
short_term = []
long_term = []

for table_cnt, table in enumerate(page_soup.find_all('table')):  

    tag = table.find_parent("div", {"class": "para-content"})
    heading = tag.find("h3").text

    payable_date=(str.strip(tag.find("p").text))
    substring_with_date = payable_date.find('Payable Date:') #stores the index of a substring or char        
    stringdata = payable_date[substring_with_date:]
    payable_date = stringdata[14:]
    
    # Only focus on final quarter distributions
    if int(payable_date[:2]) < 10:
        continue

    rows = table.findAll('tr')
    for row_cnt, tr in enumerate(rows):
        if(row_cnt !=0 ):
            distribution_date.append(datetime.date.strftime(parse(payable_date), '%m/%d/%y'))
            cols = tr.findAll('td')
            for i,td in enumerate(cols):
                if(i == 0):
                    fund_name.append(f'{heading} {td.text}')
                elif(i == 2):
                    short_term.append(float(td.text))
                elif(i == 4):
                    long_term.append(float(td.text))

#### Create DataFrame

In [24]:
df = pd.DataFrame({'Fund Name': fund_name})

df['Fund Family'] = fund_family
df['Ticker'] = None
df['Short Term Capital Gain'] = short_term
df['Long Term Capital Gain'] = long_term
df['Capital Gain Format'] = 'Per Share Percent'
df['Distribution Date'] = distribution_date
df['Timestamp'] = retrieval_timestamp

df = df[columns]
df_final = df_final.append(df).reset_index(drop=True)

df.head()

,Fund Family,Fund Name,Ticker,Short Term Capital Gain,Long Term Capital Gain,Capital Gain Format,Distribution Date,Timestamp
0,BlackRock,iShares US Aggregate Bond Index Fund Class K S...,None,0.0,0.0,Per Share Percent,11/01/21,2021-12-02 20:14:39.368997
1,BlackRock,iShares US Aggregate Bond Index Fund Instituti...,None,0.0,0.0,Per Share Percent,11/01/21,2021-12-02 20:14:39.368997
2,BlackRock,iShares US Aggregate Bond Index Fund Investor ...,None,0.0,0.0,Per Share Percent,11/01/21,2021-12-02 20:14:39.368997
3,BlackRock,iShares US Aggregate Bond Index Fund Class P S...,None,0.0,0.0,Per Share Percent,11/01/21,2021-12-02 20:14:39.368997
4,BlackRock,BlackRock Equity Dividend Fund Investor A Shares,None,0.0,0.0,Per Share Percent,10/08/21,2021-12-02 20:14:39.368997


### Columbia Management

#### _Download PDF_

In [25]:
pdf_url = 'https://www.columbiathreadneedleus.com/binaries/content/assets/cti/public/2021_cap_gain_yearend_estimates.pdf'
pdf_downloaded = download_file(pdf_url)
retrieval_timestamp = datetime.datetime.now()

#### _Parse & Collect Data_

In [26]:
fund_family = 'Columbia Management'
capital_gain_format = 'Per Share'

def columbia_data_extractor(text):
    
    output = []
    prefix = None
    
    for i, line in enumerate(text.split('\n')):
        
        if prefix != None:
            line = prefix + line
        
        if bool(re.search(r'^Columbia', line))==True and bool(re.search(r'12/\d\d/21', line))==False:
            prefix = line.strip() + ' '
            continue
        else:
            prefix = None
        
        if re.search('(^Columbia|^Multi-Manager)', line) and re.search(r'(\bFund\b|\bPortfolio\b|\bAcorn\b)', line):
            line_list = line.split()
            
            try:
                fund_name_ticker = re.search('.+X\**\s', line).group(0).strip()
            except AttributeError:
                print(f'ERROR: {line}')
                continue
            
            fund_name = ' '.join(fund_name_ticker.split()[0:-1]).strip()
            ticker = fund_name_ticker.split()[-1].replace('*','').strip()
            
            record_date = line_list[-16]
            distribution_date = line_list[-15]
            
            short_term_min = float(line_list[-14][1:]) if re.search('\d+', line_list[-14]) else np.NaN
            short_term_max = float(line_list[-13][1:]) if re.search('\d+', line_list[-13]) else np.NaN
            short_term_avg = (short_term_min + short_term_max) / 2
            
            long_term_min = float(line_list[-12][1:]) if re.search('\d+', line_list[-12]) else np.NaN
            long_term_max = float(line_list[-11][1:]) if re.search('\d+', line_list[-11]) else np.NaN
            long_term_avg = (long_term_min + long_term_max) / 2
            
            output.append([fund_family, fund_name, ticker, \
                           short_term_avg, \
                           long_term_avg, \
                           capital_gain_format, \
                           distribution_date, retrieval_timestamp])
    return output

output_data = []
with pdfplumber.open(os.path.join(data_pdf_path, pdf_downloaded)) as pdf:
    for page_num in range(len(pdf.pages)):
        page = pdf.pages[page_num]
        text = page.extract_text()
        output_data.extend(columbia_data_extractor(text))

#### _Create DataFrame_

In [27]:
df = pd.DataFrame(output_data, columns=columns)[columns]
df_final = df_final.append(df).reset_index(drop=True)

df.head()

,Fund Family,Fund Name,Ticker,Short Term Capital Gain,Long Term Capital Gain,Capital Gain Format,Distribution Date,Timestamp
0,Columbia Management,Columbia Acorn European Fund,CAEZX,0.025,0.025,Per Share,12/20/21,2021-12-02 20:14:45.414399
1,Columbia Management,Columbia Acorn Fund,ACRNX,0.085,2.750,Per Share,12/14/21,2021-12-02 20:14:45.414399
2,Columbia Management,Columbia Acorn International,ACINX,0.095,3.750,Per Share,12/20/21,2021-12-02 20:14:45.414399
3,Columbia Management,Columbia Acorn International Select,ACFFX,NaN,1.375,Per Share,12/20/21,2021-12-02 20:14:45.414399
4,Columbia Management,Columbia Acorn USA,AUSAX,0.250,4.000,Per Share,12/14/21,2021-12-02 20:14:45.414399


### Dodge & Cox

#### _Download PDF_

In [28]:
pdf_url = 'https://www.dodgeandcox.com/pdf/shareholder_services/dc_estimated_year_end_distributions_2021.pdf'

pdf_downloaded = download_file(pdf_url)
retrieval_timestamp = datetime.datetime.now()

#### Parse & Collect Data

In [29]:
fund_family = 'Dodge & Cox'
fund_info = []
distribution_date = ''

with pdfplumber.open(os.path.join(data_pdf_path, pdf_downloaded)) as pdf:
    page = pdf.pages[0]
    text = page.extract_text()
    for i, line in enumerate(text.split('\n')):
        if i == 9:
            distribution_date = parse(line.split(':')[1].strip())
            distribution_date = datetime.date.strftime(distribution_date, '%m/%d/%y')
        if i < 25 or i > 38 or len(line.strip()) == 0:
            pass
        else:
            fund_info.append(line)

fund_info[4], fund_info[5] = fund_info[5], fund_info[4]
fund_info[3:6] = [''.join(fund_info[3:6])]

fund_name = []
ticker = []
short_term = []
long_term = []

for i, line in enumerate(fund_info):
    fund_name.append(line.split('(')[0].strip())
    ticker.append(line.split('(')[1].split(')')[0].strip())
    short_term.append(float(line.split()[-4].replace('$', '')) if line.split()[-3] != 'None' else np.NaN)
    long_term.append(float(line.split()[-2].replace('$', '')) if line.split()[-1] != 'None' else np.NaN)

#### _Create DataFrame_

In [30]:
df = pd.DataFrame({'Fund Name': fund_name})
df['Fund Family'] = fund_family
df['Ticker'] = ticker
df['Short Term Capital Gain'] = short_term
df['Long Term Capital Gain'] = long_term
df['Capital Gain Format'] = 'Per Share'
df['Distribution Date'] = distribution_date
df['Timestamp'] = datetime.datetime.now()

df = df[columns]
df_final = df_final.append(df).reset_index(drop=True)
df.head(5)

,Fund Family,Fund Name,Ticker,Short Term Capital Gain,Long Term Capital Gain,Capital Gain Format,Distribution Date,Timestamp
0,Dodge & Cox,Stock Fund,DODGX,0.21,3.31,Per Share,12/17/21,2021-12-02 20:15:08.362492
1,Dodge & Cox,Global Stock Fund,DODWX,0.10,1.23,Per Share,12/17/21,2021-12-02 20:15:08.362492
2,Dodge & Cox,International Stock Fund,DODFX,NaN,NaN,Per Share,12/17/21,2021-12-02 20:15:08.362492
3,Dodge & Cox,Emerging Markets Stock Fund,DODEX,NaN,NaN,Per Share,12/17/21,2021-12-02 20:15:08.362492
4,Dodge & Cox,Balanced Fund,DODBX,0.12,8.22,Per Share,12/17/21,2021-12-02 20:15:08.362492


### Fidelity

#### _Get Soup_

In [31]:
url = 'https://www.fidelity.com/mutual-funds/information/distributions#/?table=estimated'
xpath = '//*[@id="distributions-table-container-estimated"]'
page_soup = chrome_driver(url, xpath, scroll=True, scrolls=30, scroll_hop=2000)
retrieval_timestamp = datetime.datetime.now()

#### Parse & Collect Data

In [32]:
fund_family = 'Fidelity'

table_div = page_soup.find('div', {'id': 'distributions-table-container-estimated'})
headers = []
rows = []
for i, header in enumerate(table_div.find_all('th')):
    headers.append(header.text.strip())

for j, row in enumerate(table_div.find_all('tr')):
    current_row = []
    for k, dimension in enumerate(row.find_all('td')):
        current_row.append(dimension.text)
    if len(current_row) > 0:
        rows.append(current_row)

#### _Create DataFrame_

In [33]:
df = pd.DataFrame(rows, columns=headers)

df['Fund Family'] = fund_family
df['Ticker'] = None
df['Capital Gain Format'] = 'Per Share'
df['Timestamp'] = retrieval_timestamp

# Rename columns
new_column_names = {'Short-Term Capital Gain': columns[3],\
                    'Long-Term Capital Gain': columns[4],\
                    'Pay Date': columns[6]}
df.rename(columns=new_column_names, errors='raise', inplace=True)


df = df[columns]
df['Distribution Date'] = [datetime.date.strftime(parse(date), '%m/%d/%y') for date in df['Distribution Date']] 
df_final = df_final.append(df).reset_index(drop=True)
df.head(5)

,Fund Family,Fund Name,Ticker,Short Term Capital Gain,Long Term Capital Gain,Capital Gain Format,Distribution Date,Timestamp
0,Fidelity,Fidelity Advisor® Capital Development Fund - C...,None,0.103,0.491,Per Share,12/20/21,2021-12-02 20:16:27.420772
1,Fidelity,Fidelity Advisor® Diversified Stock Fund - Cla...,None,0.986,4.699,Per Share,12/20/21,2021-12-02 20:16:27.420772
2,Fidelity,Fidelity Advisor® International Discovery Fund...,None,0.423,3.93,Per Share,12/06/21,2021-12-02 20:16:27.420772
3,Fidelity,Fidelity Asset Manager® 20%,None,0.00,0.00,Per Share,12/31/21,2021-12-02 20:16:27.420772
4,Fidelity,Fidelity Asset Manager® 30%,None,0.00,0.00,Per Share,12/31/21,2021-12-02 20:16:27.420772


### JP Morgan

#### Download PDF

In [34]:
pdf_url = 'https://am.jpmorgan.com/content/dam/jpm-am-aem/americas/us/en/supplemental/news-and-fund-announcements/mutual-funds-capital-gains-estimates.pdf'

pdf_downloaded = download_file(pdf_url)
retrieval_timestamp = datetime.datetime.now()

#### Parse & Collect Data

In [35]:
fund_family = 'JP Morgan'

output_data = []
num = 0
df = pd.DataFrame(columns=['Fund Family','Fund Name', 'Ticker', 'Short-Term Capital Gain', 'Long-Term Capital Gain', 'Pay Date', 'Timestamp'])

with pdfplumber.open(os.path.join(data_pdf_path, pdf_downloaded)) as pdf:
    for page_num in range(len(pdf.pages)):
        page = pdf.pages[page_num]
        text = page.extract_text()
        
        for l in text.split('\n'):
            if bool(re.search(r'12/\d\d/2021', l))==False:
                num += 1
                continue
            
            temp_data = l.split(' ')
            
            family = fund_family
            name = ' '.join(temp_data[:-10])
            ticker = temp_data[-10:-9]
            short = temp_data[-3:-2]
            long = temp_data[-2:-1]
            gain = 'Per Share'
            date = temp_data[-4:-3]
            time = datetime.datetime.now()
            
            row = {'Fund Family': family,\
                   'Fund Name':name,'Ticker': ticker[0],\
                   'Short Term Capital Gain': float(short[0]),\
                   'Long Term Capital Gain': float(long[0]),\
                   'Capital Gain Format': gain,\
                   'Distribution Date': datetime.date.strftime(parse(date[0]), '%m/%d/%y'),\
                   'Timestamp': time}
            
            output_data.append(row)

#### Create DataFrame

In [36]:
df = pd.DataFrame(output_data)[columns]
df_final = df_final.append(df).reset_index(drop=True)
df.head(5)

,Fund Family,Fund Name,Ticker,Short Term Capital Gain,Long Term Capital Gain,Capital Gain Format,Distribution Date,Timestamp
0,JP Morgan,Core Bond Fund,PGBOX,0.00,0.09,Per Share,12/14/21,2021-12-02 20:16:49.344027
1,JP Morgan,Core Focus SMA Fund,JCKUX,0.01,0.06,Per Share,12/14/21,2021-12-02 20:16:49.344027
2,JP Morgan,Core Plus Bond Fund,ONIAX,0.01,0.05,Per Share,12/14/21,2021-12-02 20:16:49.344027
3,JP Morgan,Corporate Bond Fund,CBRAX,0.20,0.09,Per Share,12/14/21,2021-12-02 20:16:49.344027
4,JP Morgan,Diversified Fund,JDVAX,0.74,1.84,Per Share,12/16/21,2021-12-02 20:16:49.344027


## Final Data Output

### Previews & Sanity Checks

#### Count of Funds by Fund Family

In [37]:
df_final.groupby('Fund Family')['Fund Name'].nunique()

Fund Family
American Century        95
American Funds          51
BlackRock              170
Columbia Management    100
Dodge & Cox              7
Fidelity               423
Franklin Templeton     184
JP Morgan               70
PIMCO                   76
T. Rowe Price          276
Name: Fund Name, dtype: int64

#### Check for Duplicate Fund Names (Should Return Empty Series)

In [38]:
pd.set_option('display.max_colwidth', None)
df_dupes = df_final[df_final.duplicated(subset=['Fund Family', 'Fund Name'])]
df_dupes['Fund Name'].sort_values(ascending=True)

Series([], Name: Fund Name, dtype: object)

#### Preview Final DataFrame

In [39]:
df_final

,Fund Family,Fund Name,Ticker,Short Term Capital Gain,Long Term Capital Gain,Capital Gain Format,Distribution Date,Timestamp
0,T. Rowe Price,T. Rowe Price Domestic Stock Funds - Dividend Growth (PRDGX),PRDGX,NaN,0.89,Per Share,12/14/21,2021-12-02 20:12:13.973473
1,T. Rowe Price,T. Rowe Price Domestic Stock Funds - Equity Income (PRFDX),PRFDX,0.36,1.95,Per Share,12/14/21,2021-12-02 20:12:13.973473
2,T. Rowe Price,T. Rowe Price Domestic Stock Funds - Equity Index 500 (PREIX),PREIX,0.07,0.47,Per Share,12/14/21,2021-12-02 20:12:13.973473
3,T. Rowe Price,T. Rowe Price Domestic Stock Funds - Extended Equity Market Index (PEXMX),PEXMX,NaN,5.06,Per Share,12/14/21,2021-12-02 20:12:13.973473
4,T. Rowe Price,T. Rowe Price Domestic Stock Funds - Real Estate (TRREX),TRREX,NaN,2.8,Per Share,12/14/21,2021-12-02 20:12:13.973473
...,...,...,...,...,...,...,...,...
1447,JP Morgan,U.S. Small Company Fund,JTUAX,1.98,2.75,Per Share,12/14/21,2021-12-02 20:16:49.533551
1448,JP Morgan,U.S. Sustainable Leaders Fund,JICAX,0.96,1.43,Per Share,12/14/21,2021-12-02 20:16:49.533551
1449,JP Morgan,U.S. Value Fund,VGRIX,0.5,0.96,Per Share,12/14/21,2021-12-02 20:16:49.533551
1450,JP Morgan,Undiscovered Managers Behavioral Value Fund,UBVAX,0.21,1.33,Per Share,12/14/21,2021-12-02 20:16:49.533551


#### Output Data to Excel

In [40]:
df_final.to_excel(f'schedule.xlsx',index=False)